In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install tensorflow
! pip install numpy==1.21.6

In [2]:
import pandas as pd
import numpy as np
import random
import os
import keras

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [3]:
# 시드 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [4]:
train = pd.read_csv('/content/drive/MyDrive/kiwom/train.csv')
train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


# prophet 모델

In [ ]:
!pip install --upgrade plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [ ]:
%pip install prophet

In [ ]:
! python -m pip install Prophet

In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

In [5]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [7]:
## prophet 모델
from prophet import Prophet
import matplotlib.pyplot as plt
import matplotlib
import itertools
#import talib
#import pmdarima as pm

import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
import warnings
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

#################

train = pd.read_csv('/content/drive/MyDrive/kiwom/train.csv')

# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame()

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()


# fig, axes = plt.subplots(5,5,figsize =(20,20))
# cnt = 0

## 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
#code = "A103840"
    # 20일선 가격의 상승분과 비교
    train_close = train[train['종목코드'] == code][['일자', '종가', '종목코드']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index(np.arange(len(train_close)), inplace=True)
    train_close.columns = ['일자', '종가','종목코드']

    # n일 이동평균선
    train_close["MA5_mean"]=train_close["종가"].rolling(5).mean()
    train_close["MA10_mean"]=train_close["종가"].rolling(10).mean()
    train_close["MA20_mean"]=train_close["종가"].rolling(20).mean()
    train_close["MA60_mean"]=train_close["종가"].rolling(60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    reg = []
    rev = []

    for i in range(len(train_close)):
        if train_close["MA5_mean"].iloc[i] > train_close["MA10_mean"].iloc[i] and train_close["MA10_mean"].iloc[i] > train_close["MA20_mean"].iloc[i] and train_close["MA20_mean"].iloc[i] > train_close["MA60_mean"].iloc[i]:
            reg.append(1)
            rev.append(0)
        elif train_close["MA5_mean"].iloc[i] < train_close["MA10_mean"].iloc[i] and train_close["MA10_mean"].iloc[i] < train_close["MA20_mean"].iloc[i] and train_close["MA20_mean"].iloc[i] < train_close["MA60_mean"].iloc[i]:
            rev.append(1)
            reg.append(0)
        else:
            reg.append(0)
            rev.append(0)

    train_close["regular"] = pd.Series(reg)
    train_close["reverse"] = pd.Series(rev)

    train_close["reg_sum"] = train_close["regular"].rolling(20).sum()
    train_close["rev_sum"] = train_close["reverse"].rolling(20).sum()
    # train_close["MA20-MA60"] = (train_close["MA20_mean"]-train_close["MA60_mean"])/train_close["MA60_mean"]*50

    # 이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    train_close["MA5-MA10"] = (train_close["MA5_mean"]-train_close["MA10_mean"])/train_close["MA10_mean"]*100
    train_close["score"] = (train_close["reg_sum"]-train_close["rev_sum"]+train_close["MA5-MA10"])

    #가중치
    tc = train_close[["일자","score"]]

    #model = prophet()
    future_df = tc.copy()
    future_df.rename(columns={'일자':'ds','score':'y'},inplace=True)

    m = Prophet(seasonality_mode='additive') # (yearly_seasonality=True, daily_seasonality=True)
    m.fit(future_df)
    future = m.make_future_dataframe(periods=15)
    forecast = m.predict(future)
    forecasted = list(np.array(forecast['yhat'].tail(15))) # 예측 데이터 리스트
    #forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

    # 최종 수익률 계산
    #final_return = predictions.iloc[-1]
    final_return = forecasted[-1]

    # 가장 마지막에 있는 것의 배열상태에 따라 순위 매긴다.
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

results_df

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 77%|███████▋  | 1544/2000 [07:58<02:04,  3.67it/s]INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 77%|███████▋  | 1545/2000 [07:58<01:55,  3.94it/s]INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcyqe7klu/mchh_5ui.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcyqe7klu/i86p39zv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=72647', 'data', 'file=/tmp/tmpcyqe7klu/mchh_5ui.json', 'init=/tmp/tmpcyqe7klu/i86p39zv.json', 'output', 'file=/tmp/tmpcyqe7klu/proph

,종목코드,final_return
0,A060310,-7.158594
1,A095570,-19.315971
2,A006840,3.216481
3,A054620,-3.277627
4,A265520,9.386337
...,...,...
1995,A189980,-1.130665
1996,A000540,-4.354501
1997,A003280,24.057053
1998,A037440,-1.270456


In [8]:
results_df # prophet 전체 final_retrun

,종목코드,final_return
0,A060310,-7.158594
1,A095570,-19.315971
2,A006840,3.216481
3,A054620,-3.277627
4,A265520,9.386337
...,...,...
1995,A189980,-1.130665
1996,A000540,-4.354501
1997,A003280,24.057053
1998,A037440,-1.270456


In [9]:
#results_df['final_return'] = results_df['final_return'].fillna(0)
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df = results_df.sort_values(by='순위')

sample_submission = pd.read_csv('/content/drive/MyDrive/kiwom/sample_submission.csv')
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='right')
baseline_submission.to_csv('/content/drive/MyDrive/kiwom/0726_MA_test_submission.csv', index=False)
results_df

,종목코드,final_return,순위
1494,A051980,28.254024,1
1131,A064850,28.208472,2
1128,A247540,25.857006,3
1160,A058970,25.795445,4
1590,A009730,25.307356,5
...,...,...,...
789,A038540,-23.723253,1996
1979,A090710,-24.434550,1997
902,A013810,-24.468925,1998
1056,A030960,-30.996631,1999


In [10]:
len(forecasted)

15